# Tutorial 1: Generalization in AI

**Week 1, Day 1: Generalization**

**By Neuromatch Academy**

__Content creators:__ Names & Surnames

__Content reviewers:__ Names & Surnames

__Production editors:__ Names & Surnames

<br>

Acknowledgments: [ACKNOWLEDGMENT_INFORMATION]


___


# Tutorial Objectives

*Estimated timing of tutorial: [insert estimated duration of whole tutorial in minutes]*

In this tutorial, you will investigate the problem of generalization in artificial intelligence, neuroscience and cognitive science. 


In [ ]:
# @title Tutorial slides
# @markdown These are the slides for the videos in all tutorials today

## Uncomment the code below to test your function

#from IPython.display import IFrame
#link_id = "<YOUR_LINK_ID_HERE>"

print("If you want to download the slides: 'Link to the slides'")
      # Example: https://osf.io/download/{link_id}/

#IFrame(src=f"https://mfr.ca-1.osf.io/render?url=https://osf.io/download/{link_id}/?direct%26mode=render", width=854, height=480)

---
# Setup



In [ ]:
# @title Install dependencies
# @markdown

#!pip install numpy Pillow matplotlib torch torchvision transformers gradio sentencepiece protobuf
#!pip install git+https://github.com/Belval/TextRecognitionDataGenerator#egg=trdg

In [ ]:
# @title Import dependencies
# @markdown

# Standard Libraries for file and operating system operations, security, and web requests
import os
import hashlib
import requests
import logging
import io

# Core python data science and image processing libraries
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Deep Learning and model specific libraries
import torch
import transformers
from torchvision import transforms
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import google.protobuf

# Utility and interface libraries
import gradio as gr
from IPython.display import IFrame
import trdg
from trdg.generators import GeneratorFromStrings
import sentencepiece
import zipfile

In [ ]:
# @title Figure settings
# @markdown

logging.getLogger('matplotlib.font_manager').disabled = True

%matplotlib inline
%config InlineBackend.figure_format = 'retina' # perfrom high definition rendering for images and plots
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/main/nma.mplstyle")

In [ ]:
# @title Plotting functions
# @markdown

def display_image(image_path):
    """Display an image from a given file path.

    Inputs:
    - image_path (str): The path to the image file.
    """
    # Open the image
    image = Image.open(image_path)
    if image.mode != 'RGB':
        image = image.convert('RGB')

    # Display the image
    plt.imshow(image)
    plt.axis('off')  # Turn off the axis
    plt.show()

def display_transformed_images(image, transformations):
    """
    Apply a list of transformations to an image and display them.

    Inputs:
    - image (Tensor): The input image as a tensor.
    - transformations (list): A list of torchvision transformations to apply.
    """
    # Convert tensor image to PIL Image for display
    pil_image = transforms.ToPILImage()(image)

    fig, axs = plt.subplots(len(transformations) + 1, 1, figsize=(5, 15))
    axs[0].imshow(pil_image)
    axs[0].set_title('Original')
    axs[0].axis('off')

    for i, transform in enumerate(transformations):
        # Apply transformation if it's not the placeholder
        if transform != "Custom ElasticTransform Placeholder":
            transformed_image = transform(image)
            # Convert transformed tensor image to PIL Image for display
            display_image = transforms.ToPILImage()(transformed_image)
            axs[i+1].imshow(display_image)
            axs[i+1].set_title(transform.__class__.__name__)
            axs[i+1].axis('off')
        else:
            axs[i+1].text(0.5, 0.5, 'ElasticTransform Placeholder', ha='center')
            axs[i+1].axis('off')

    plt.tight_layout()
    plt.show()

def display_original_and_transformed_images(original_tensor, transformed_tensor):
    """
    Display the original and transformed images side by side.

    Inputs:
    - original_tensor (Tensor): The original image as a tensor.
    - transformed_tensor (Tensor): The transformed image as a tensor.
    """
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    # Display original image
    original_image = original_tensor.permute(1, 2, 0)  # Convert from (C, H, W) to (H, W, C)
    axs[0].imshow(original_image)
    axs[0].set_title('Original')
    axs[0].axis('off')

    # Display transformed image
    transformed_image = transformed_tensor.permute(1, 2, 0)  # Convert from (C, H, W) to (H, W, C)
    axs[1].imshow(transformed_image)
    axs[1].set_title('Transformed')
    axs[1].axis('off')

    plt.show()

def display_generated_images(generator):
    """
    Display images generated from strings.

    Inputs:
    - generator (GeneratorFromStrings): A generator that produces images from strings.
    """
    plt.figure(figsize=(15, 3))
    for i, (text_img, lbl) in enumerate(generator, 1):
        ax = plt.subplot(1, len(generator.strings) * generator.count // len(generator.strings), i)
        plt.imshow(text_img)
        plt.title(f"Example {i}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
# @title Data retrieval
# @markdown

def download_file(fname, url, expected_md5):
    """
    Downloads a file from the given URL and saves it locally.

    Inputs:
    - fname (str): The local filename/path to save the downloaded file.
    - url (str): The URL from which to download the file.
    - expected_md5 (str): The expected MD5 checksum to verify the integrity of the downloaded data.
    """
    if not os.path.isfile(fname):
        try:
            # Attempt to download the file
            r = requests.get(url)  # Make a GET request to the specified URL
        except requests.ConnectionError:
            # Handle connection errors during the download
            print("!!! Failed to download data !!!")
        else:
            # No connection errors, proceed to check the response
            if r.status_code != requests.codes.ok:
                # Check if the HTTP response status code indicates a successful download
                print("!!! Failed to download data !!!")
            elif hashlib.md5(r.content).hexdigest() != expected_md5:
                # Verify the integrity of the downloaded file using MD5 checksum
                print("!!! Data download appears corrupted !!!")
            else:
                # If download is successful and data is not corrupted, save the file
                with open(fname, "wb") as fid:
                    fid.write(r.content)  # Write the downloaded content to a file
                print(f"{fname} has been downloaded successfully.")

# Variables for the font file and download URL
fname = "Dancing_Script.zip"
url = "https://osf.io/32yed/download"
expected_md5 = "d59bd3201b58a37d0d3b4cd0b0ec7400"

# Download the font file
download_file(fname, url, expected_md5)

def extract_zip(zip_fname):
    """
    Extracts a ZIP file to the current directory.

    Inputs:
    - zip_fname (str): The filename/path of the ZIP file to be extracted.
    """
    with zipfile.ZipFile(zip_fname, 'r') as zip_ref:
        zip_ref.extractall(".")

# Extract the downloaded ZIP file
extract_zip(fname)

def download_image(fname, url, expected_md5):
    """
    Downloads an image file from the given URL and saves it locally.

    Inputs:
    - fname (str): The local filename/path to save the downloaded image.
    - url (str): The URL from which to download the image.
    - expected_md5 (str): The expected MD5 checksum to verify the integrity of the downloaded data.
    """
    if not os.path.isfile(fname):
        try:
            # Attempt to download the file
            r = requests.get(url) # Make a GET request to the specified URL
        except requests.ConnectionError:
            # Handle connection errors during the download
            print("!!! Failed to download data !!!")
        else:
            # No connection errors, proceed to check the response
            if r.status_code != requests.codes.ok:
                # Check if the HTTP response status code indicates a successful download
                print("!!! Failed to download data !!!")
            elif hashlib.md5(r.content).hexdigest() != expected_md5:
                # Verify the integrity of the downloaded file using MD5 checksum
                print("!!! Data download appears corrupted !!!")
            else:
                # If download is successful and data is not corrupted, save the file
                with open(fname, "wb") as fid:
                    fid.write(r.content) # Write the downloaded content to a file

# Variables for file and download URL
fnames = ["img_1235.jpg", "image_augmentation.png"]  # The names of the files to be downloaded
urls = ["https://osf.io/kv5bx/download", "https://osf.io/fqwsr/download"]  # URLs from where the files will be downloaded
expected_md5s = ["920ae567f707bfee0be29dc854f804ed", "f4f1ebee1470a7e2d7662eec1d193ba2"] # MD5 hashes for verifying files integrity

for fname, url, expected_md5 in zip(fnames, urls, expected_md5s):
    download_image(fname, url, expected_md5)


# Generalization in AI

## Tutorial-specific learning Objective: 

1. Describe some common goals of developers of deployed AI systems:
- Performance, latency, SWaP-C
- Explanation/understanding
2. Describe (at least) 3 ways of tackling generalization with AI, including the most currently popular trends of fitting generic models on large-scale datasets (i.e. bitter lesson)
3. Get hands-on experience in the basics of deep learning and PyTorch


## Video and background

### Missing

## Activity 1: Introducing the TrOCR (Transformer-based Optical Character Recognition) Model
In this section, we introduce TrOCR, a transformer-based model developed by Microsoft, designed for Optical Character Recognition (OCR). The goal of TrOCR is to accurately convert images of text into machine-readable text. This model is an example of how AI can bridge the gap between visual data and text data.

### Loading 

We start by loading the pre-trained TrOCR model and its associated processor. The model VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-small-handwritten") is specifically trained to recognize handwritten text, while the TrOCRProcessor is used for preparing the input images and decoding the model's predictions into human-readable text. This demonstrates the ease of accessing and utilizing advanced machine learning models with just a few lines of code.


In [ ]:
# Load the pre-trained TrOCR model and processor
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-small-handwritten")
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-small-handwritten")


Okay, the model is loaded! Now, we need a "recognize_text" function. This is a crucial component that showcases how to apply the loaded model to perform OCR on an input image. It uses a pre-trained model for Optical Character Recognition (OCR) to convert an image to text. It preprocesses the image to the model's input specifications, predicts the text as token IDs, decodes these tokens into human-readable text while skipping special tokens, and returns the recognized text.

In [ ]:
# Define the function to recognize text from an image
def recognize_text(image):
    """
    This function takes an image as input and uses a pre-trained language model to generate text from the image.

    Inputs:
    - image (PIL Image or Tensor): The input image containing text to be recognized.

    Outputs:
    - text (str): The recognized text extracted from the input image.
    """
    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)
    text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return text

Now, we employ Gradio to create an interactive web interface for our TrOCR demo. Gradio is a Python library that simplifies creating UIs for machine learning models. This interface allows users to upload images and see the recognized text, demonstrating the model's capabilities interactively. 

In [ ]:
# Create a Gradio interface
interface = gr.Interface(
    fn=recognize_text,
    inputs=gr.Image(type="pil"),
    outputs=gr.Textbox(),
    title="Interactive demo: TrOCR",
    description="Demo for Microsoft’s TrOCR, an encoder-decoder model for OCR on single-text line images.",
)

# Launch the interface
interface.launch()

## Activity 2: Inspecting the Model's Encoder and Decoder
By inspecting the model's encoder and decoder, we gain insight into the inner workings of the TrOCR model. The encoder processes the input images, and the decoder generates text predictions. This step is crucial for understanding the model architecture and the roles of its components in the text recognition process. Try it out!

In [ ]:
# Inspect the encoder of the model
model.encoder

### Coding exercise

In the cell below, inspect the decoder of the model!

In [ ]:
###

### Follow-up coding exercise 

Now, we want to create a count_parameters function that calculates the total number of parameters in the model, providing a sense of the model's complexity. Counting parameters in the encoder and decoder separately helps appreciate the distribution of model complexity and its implications on training and inference.

In [ ]:
# Function to count the parameters of the model
def count_parameters(model):
    """
    This function calculates the total number of parameters in a given PyTorch model.

    Inputs:
    - model (torch.nn.Module): The PyTorch model for which parameters are to be counted.

    Outputs:
    - num_parameters (int): The total number of parameters in the specified model.
    """
    return sum(p.numel() for p in model.parameters())

# Count parameters in the encoder
encoder_params = count_parameters(model.encoder)

# Count parameters in the decoder
decoder_params = count_parameters(model.decoder)

encoder_params, decoder_params

### Written exercise

Look at the available variants of this model (i.e. small, base, large). How do these models trade off in terms of accuracy vs. inference latency? 

### Discussion point

What kinds of inductive biases are embedded inside a model such as this one?

## Activity 3: generalization, part I: transfer learning 

In this section, we take a look at how much data is distilled inside the model, focusing on the decoder. We want to calculate how long it would take to write a certain number of words, comparing human writing capabilities with the model's text generation speed. This will prompt us to discuss the efficiency of machine learning models in automating tasks traditionally performed by humans. 

In [ ]:
def calculate_writing_time(total_words, words_per_day, days_per_week, weeks_per_year, average_human_lifespan):
    """
    Calculate the time required to write a given number of words in lifetimes.

    Inputs:
    - total_words (int): total number of words to be written.
    - words_per_day (int): number of words written per day.
    - days_per_week (int): number of days dedicated to writing per week.
    - weeks_per_year (int): number of weeks dedicated to writing per year.
    - average_human_lifespan (int): average lifespan of a human in years.

    Outpus:
    - time_to_write_lifetimes (float): time to write the given words in lifetimes.
    """

    #################################################
    ## TODO for students: fill in the missing variables ##
    # Fill out function and remove
    raise NotImplementedError("Student exercise: fill in the missing variables")
    #################################################

    words_per_year = words_per_day * days_per_week * weeks_per_year

    # Calculate the time to write in years
    time_to_write_years = total_words / ...

    # Calculate the time to write in lifetimes
    time_to_write_lifetimes = time_to_write_years / average_human_lifespan

    return time_to_write_lifetimes

# Example values
total_words = 5e9
words_per_day = 1500
days_per_week = 6
weeks_per_year = 50
average_human_lifespan = 80

# Uncomment the code below to test your function

# Test the function
#time_to_write_lifetimes_roberta = calculate_writing_time(
    #total_words,
    #words_per_day,
    #days_per_week,
    #weeks_per_year,
    #average_human_lifespan
#)

# Print the result
#print(f"Time to write {total_words} words in lifetimes: {time_to_write_lifetimes_roberta} lifetimes")

In [ ]:
# to_remove solution

def calculate_writing_time(total_words, words_per_day, days_per_week, weeks_per_year, average_human_lifespan):
    """
    Calculate the time required to write a given number of words in lifetimes.

    Inputs:
    - total_words (int): total number of words to be written.
    - words_per_day (int): number of words written per day.
    - days_per_week (int): number of days dedicated to writing per week.
    - weeks_per_year (int): number of weeks dedicated to writing per year.
    - average_human_lifespan (int): average lifespan of a human in years.

    Outpus:
    - time_to_write_lifetimes (float): time to write the given words in lifetimes.
    """

    words_per_year = words_per_day * days_per_week * weeks_per_year

    # Calculate the time to write in years
    time_to_write_years = total_words / words_per_year

    # Calculate the time to write in lifetimes
    time_to_write_lifetimes = time_to_write_years / average_human_lifespan

    return time_to_write_lifetimes

# Example values
total_words = 5e9
words_per_day = 1500
days_per_week = 6
weeks_per_year = 50
average_human_lifespan = 80

# Uncomment the code below to test your function

# Test the function
#time_to_write_lifetimes_roberta = calculate_writing_time(
    #total_words,
    #words_per_day,
    #days_per_week,
    #weeks_per_year,
    #average_human_lifespan
#)

# Print the result
#print(f"Time to write {total_words} words in lifetimes: {time_to_write_lifetimes_roberta} lifetimes")

### Exploring LLaMA 2

RoBERTa is a pretty tiny model by modern standards. A more modern LLM like Llama 2 is trained on 2 trillion tokens. How much time would it take to generate that much text?

In [ ]:
#Exploring Llama 2
total_tokens_llama2 = 2e12
total_words_llama2 = 2e12 / 1.5 #assuming 1.5 words per token

In [ ]:
# Time to generate text
time_to_write_lifetimes_llama = calculate_writing_time(total_words_llama2, words_per_day, days_per_week, weeks_per_year, average_human_lifespan)

Assuming 1.5 tokens/word, it would take ~=37,000 lifetimes!

## Activity 3: generalization, part II: augmentation

This section introduces the concept of image augmentation, a technique used to increase the diversity of training data and improve model generalization. When data is not abundant, we can improve generalization by augmenting the existing dataset with variants of the same data. Thus, we take an expressive model with few built-in inductive biases, and through demonstrations, let it learn invariances and equivariances in the data, encouraging generalization.

By applying various transformations to images and displaying the results, you can visually understand how augmentation works and its impact on model performance. 

Let's start with loading and visualizing our chosen image.

In [ ]:
# Usage
image_path = 'img_1235.jpg'
display_image(image_path)

Now, we will apply a few transformations to this image. You can play around with the input values!

In [ ]:
# Convert PIL Image to Tensor
image = Image.open(image_path)
image = transforms.ToTensor()(image)

# Define each transformation separately
# RandomAffine: applies rotations, translations, scaling. Here, rotates by up to ±15 degrees,
affine = transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1))

# ElasticTransform: applies elastic distortions to the image. The 'alpha' parameter controls
# the intensity of the distortion.
elastic = transforms.ElasticTransform(alpha=50.0)

# RandomPerspective: applies random perspective transformations with a specified distortion scale.
perspective = transforms.RandomPerspective(distortion_scale=0.2, p=1.0)

# RandomErasing: randomly erases a rectangle area in the image.
erasing = transforms.RandomErasing(p=1, scale=(0.02, 0.33), ratio=(0.3, 3.3), value='random', inplace=False)

# GaussianBlur: applies gaussian blur with specified kernel size and sigma range.
gaussian_blur = transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.4, 5))

Let's now combine them in a single list and display the image.

In [ ]:
# A list of all transformations for iteration
transformations = [affine, elastic, perspective, erasing, gaussian_blur]

# Display
display_transformed_images(image, transformations)

We can also use transforms.Compose from PyTorch to apply all of these transformations simultaneously!

In [ ]:
# Path to the image
image_path = 'image_augmentation.png'

# Open the image
image = Image.open(image_path)

# Convert PIL Image to Tensor
image_tensor = transforms.ToTensor()(image)

# Define transformations here
affine = transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1))
elastic = transforms.ElasticTransform(alpha=90.0)
perspective = transforms.RandomPerspective(distortion_scale=0.2, p=1.0)
erasing = transforms.RandomErasing(p=1, scale=(0.02, 0.33), ratio=(0.3, 3.3), value='random', inplace=False)
gaussian_blur = transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))

# Combine all the transformations
all_transforms = transforms.Compose([
    affine,
    elastic,
    perspective,
    erasing,
    gaussian_blur
])

# Apply combined transformation
augmented_image_tensor = all_transforms(image_tensor)

display_original_and_transformed_images(image_tensor, augmented_image_tensor)

## Activity 3: generalization, part III: synthetic data

When augmentation is not enough, we can further improve generalization by training on synthetic data. This allows us to stretch our data even further. Here, we will define strings and create a generator to generate a synthetic version of the input data.

In [ ]:
# Define your strings
strings = ['Hello', 'This is Patrick', 'From NMA']

# Specify font path
font_path = "DancingScript-VariableFont_wght.ttf"  # Ensure this path is correct

# Create a generator with the specified parameters
generator = GeneratorFromStrings(
    strings=strings,
    fonts=[font_path],
    space_width=2,
    skewing_angle=8,
    count=3
)

# Define the desired size
desired_size = (500, 300)  # Width, Height in pixels

# Function to resize images
def resize_image(image, new_size):
    return image.resize(new_size, Image.Resampling.LANCZOS)

for img, lbl in generator:
    # Resize the image before showing it
    img = resize_image(img, desired_size)
    img.show()

# Call the function with the generator
display_generated_images(generator)

### Discussion point

What does this type of synthetic data capture that wouldn’t be easy to capture through data augmentation?

### Generating handwriting style data

We can take this idea further and generate handwriting style data. Use an embedded calligrapher.ai model to generate new snippets of writing data.

In [ ]:
IFrame("https://www.calligrapher.ai/", width=800, height=600)

### Discussion point

What kinds of variation does this synthetic data capture that wouldn’t be easy to reproduce to all our other data generation pipelines?